In [1]:
# Setup prior to Test
# find out what dsxhi register system to use by the project name.  Project is named with dsxhi register system name
import json
import os
_project_name_= os.getenv("PROJECT_NAME")
if (not _project_name_) :
 print("No project name found, stop!!!")
else :
 print("Project Name is: " + _project_name_ )

_project_id_=os.getenv("PROJECT_ID")

# When I created the project, I named the project prefixed with edge node short name
edge_node_name =_project_name_.split('Project',1)

dsxhi_register_name = edge_node_name[0]
print("DSXHI systerm display name is : " + dsxhi_register_name)
_user_id_= os.getenv("USER_ID")
print("USER ID is : " + _user_id_ + " , Project ID is " + _project_id_ + " and project_name is " + _project_name_)

Project Name is: rated3ProjectV30
DSXHI systerm display name is : rated3
USER ID is : 1000331002 , Project ID is ea65b3b4-173c-41f4-a6d0-3c048987c47d and project_name is rated3ProjectV30


In [2]:
# Setup prior to Test
import hadoop_lib_utils
my_image_id = ""
image_list = ""
my_Livy_endpoint = ""
found = 0
# Looking for this image ID
python_image_id ="jupyter-py36"

# Looking for livy version
livy_spark = "LIVYSPARK2"
print("Looking for livy version " + livy_spark + " in dsxhi registered display name " + dsxhi_register_name 
      + " and Image_id name " + python_image_id + "\n\n")

dsxhi_reg_info = hadoop_lib_utils.get_dsxhi_info(showSummary=True)

webhdfs_name = "WEBHDFS"
my_webhdfs_endpoint = ""
webhdfs_found = 0
for row in dsxhi_reg_info :
    # print(json.dumps(row))
    
    system = row["system"]
    
    if (system != dsxhi_register_name):
        print("Skip this system " + system)
        continue
        
    my_livy_endpoint = row[livy_spark]
    my_webhdfs_endpoint = row[webhdfs_name]

    if (my_livy_endpoint):
        # print("Found dsxhi systerm with livy spark url" + my_livy_endpoint)
        # print("Found dsxhi systerm with webhdfs url" + my_webhdfs_endpoint)
        print("my_livy_endpoint in system " + system + " livy endpoint found is: " + my_livy_endpoint)
         
        image_list = row["imageList"]
                       
        for image_id in image_list:
            # print ("Image ID is " + image_id)
            if (image_id.find(python_image_id) >= 0):
                my_image_id = python_image_id
                # print ("Found the image ID " + image_id)
                found = 1
                break                   
       
    if (found):
        break

print("\n\nDSXHI register system: " + dsxhi_register_name)                
print("Python image ID: " + my_image_id)   
print("Livy endpoint: " +  my_livy_endpoint) 
print("Webhdfs endpoint: " + my_webhdfs_endpoint) 
if (not my_livy_endpoint or not my_webhdfs_endpoint):
   print("!!!STOP!!!, require info not found !!!!")
if (not found):
    print("Image ID not found !!!!")
    

Looking for livy version LIVYSPARK2 in dsxhi registered display name rated3 and Image_id name jupyter-py36


Available Hadoop systems: 

      systemName LIVYSPARK  LIVYSPARK2       imageId
0   cdh-outsider            livyspark2  jupyter-py36
1  cdhv62-edge-1            livyspark2  jupyter-py36
2     datapuddle            livyspark2              
3     hdp31edge1            livyspark2              
4         rated3            livyspark2  jupyter-py36
5         yccdh5            livyspark2              
Skip this system cdh-outsider
Skip this system cdhv62-edge-1
Skip this system datapuddle
Skip this system hdp31edge1
my_livy_endpoint in system rated3 livy endpoint found is: https://internal-nginx-svc:12443/ibm-dsxhi-rated3/livy2/v1


DSXHI register system: rated3
Python image ID: jupyter-py36
Livy endpoint: https://internal-nginx-svc:12443/ibm-dsxhi-rated3/livy2/v1
Webhdfs endpoint: https://internal-nginx-svc:12443/ibm-dsxhi-rated3/webhdfs/v1


### In order to run Livy sessions on a remote Hadoop cluster, your Watson Studio admin must first 
### register a Hadoop Integration system with Watson Studio.

### Ask your Watson Studio admin to use the Admin Console => Hadoop Integration option to register 
### a Hadoop Integration system. NOTE: Installation and configuration of IBM's Hadoop Integration (HI) 
### service on a Hadoop cluster must be done by a Hadoop admin before that system can be registered with your Watson Studio account.

### When your admin indicates that a Hadoop Integration system has been registered, you can proceed with this sample notebook.

In [3]:
# Imports needed for the cells which run locally on Watson Studio.
# import dsx_core_utils
import hadoop_lib_utils
import pandas as pd
from sklearn.externals import joblib

/opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Configure the Spark session that we will run on the selected registered HI system. 
#### In this case we want the session to start with 1G memory and two Spark executors. 
### NOTE: myConfig here is optional; if you prefer to use default configs you can omit this cell and remove the addlConfig argument in the next cell.

In [4]:
myConfig={
 "queue": "default",
 "driverMemory": "1G",
 "numExecutors": 2
};

In [5]:
# Set up sparkmagic to connect to the selected registered HI
# system with the specified configs. **NOTE** This notebook
# requires Spark 2, so you should set 'livy' to 'livyspark2'.
# Use imageId="jupyter" if running the notebook on Python 3.5 

my_imageID = "jupyter-py35"
livy_version = "livyspark2"
print("DSXHI registered name is " + dsxhi_register_name + " Spark version is " + livy_version + " image ID is " + my_imageID)
HI_CONFIG = hadoop_lib_utils.setup_livy_sparkmagic(
  system=dsxhi_register_name, 
  livy=livy_version,
  # imageId=my_imageID,
  addlConfig=myConfig)

# (Re-)load sparkmagic to apply the new configs.
%reload_ext sparkmagic.magics

DSXHI registered name is rated3 Spark version is livyspark2 image ID is jupyter-py35
sparkmagic has been configured to use https://ibm-dsxhi-rated3-svc:8443/livy2/v1 
success configuring sparkmagic livy.


### Now, let's capture some state about the configured Hadoop Integraton system, to be used later in this notebook. 
### Then start up a new, remote Livy session to connect to that HI system. NOTE: Depending on 
### a) the resources available in the remote Hadoop system and 
### b) the speed of your cluster, attempts to start the session might report errors due to timeout or due to a session coming up dead. 
### In such cases you should run %spark cleanup as a separate cell, then re-run this cell again. 
### If session creation continues to fail, contact the Hadop admin of the target Hadoop cluster to see if everything is configured as expected.

In [6]:
session_name = 'mlsess1'
livy_endpoint = HI_CONFIG['LIVY']
webhdfs_endpoint = HI_CONFIG['WEBHDFS']
%spark add -s $session_name -l python -k -u $livy_endpoint

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2045,application_1570832601630_0125,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


### For reference / debugging: Print out the name of the Hadoop node to which the remote session has been assigned. 
### When "local" files are created within the remote session, they will be written to this node. 
### All of the Yarn container artifacts (workspace and temp files) will exist on this node, as well.
### Test spark job by getting catalog info


In [7]:
%%spark -s $session_name
import socket
print("Remote livy session driver: {}".format(socket.gethostname()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Remote livy session driver: rated6.fyre.ibm.com

### The following cell, and all subsequent cells which have %%spark as their first line, will run remotely, 
### i.e. within a Yarn container that exists on the registered Hadoop Integration system.

In [8]:
%%spark -s $session_name

# Declare imports needed for all of the cells that will run remotely.
import getpass, time, os, shutil
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

# Load IBM Hadoop Integration utilities to facilitate remote functionality.
# This line assumes that HI version >= X.Y has been installed on the registered
# Hadoop Integration system.
hi_utils_lib = os.getenv("HI_UTILS_PATH", "")
sc.addPyFile("hdfs://{}".format(hi_utils_lib))
import hi_core_utils

# Declare a target HDFS directory path that will be used for our data.
hdfs_dataset_dir = "/user/{}/datasets".format(getpass.getuser())
input_ds = "{}/{}".format(hdfs_dataset_dir, "cars.csv")

# Create target hdfs directory, if it does not already exist.
# I have this dir in my hadoop already, skip
# hi_core_utils.run_command("hdfs dfs -mkdir -p {}".format(hdfs_dataset_dir))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Now, let's load our test data into HDFS. For the purposes of this sample, our data is small and comes from the local cars.csv file created above. 
### We do not need to put it into HDFS for this example--but we choose to do so for demonstration purposes. 
###  In a real scenario the desired data should already be loaded into HDFS.


In [9]:
# Redeclare the dataset dir locally--the earlier declaration was in the _remote_
# session so it is not available here.

# ** NOTE ** Replace {your-username-here} with your actual user name.
# *** Skip the following because I have the setup in my hadoop system *** 
# hdfs_dataset_dir = "/user/user1/datasets"

# Upload the saved csv file from Local to the remote HDFS.
#input_csv = os.environ["DSX_PROJECT_DIR"] + "/datasets/cars.csv"
# input_csv = "/project_data/data_asset/cars.csv"
# hadoop_lib_utils.upload_hdfs_file(webhdfs_endpoint, input_csv, "{}/cars.csv".format(hdfs_dataset_dir))


### Access and Manipulate Data
### Now use Spark to read the data, as a Spark dataframe, from HDFS.

In [10]:
%%spark -s $session_name

df_data_0 = spark.read.format(
    "org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").option(
    "header", "true").option("inferSchema", "true").load(input_ds)

df_data_0.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------+------+----------+------+------------+----+--------+--------------------+
|mpg|cylinders|engine|horsepower|weight|acceleration|year|  origin|                name|
+---+---------+------+----------+------+------------+----+--------+--------------------+
| 18|        8| 307.0|       130|  3504|        12.0|  70|American|chevrolet chevell...|
| 15|        8| 350.0|       165|  3693|        11.5|  70|American|   buick skylark 320|
| 18|        8| 318.0|       150|  3436|        11.0|  70|American|  plymouth satellite|
| 16|        8| 304.0|       150|  3433|        12.0|  70|American|       amc rebel sst|
| 17|        8| 302.0|       140|  3449|        10.5|  70|American|         ford torino|
+---+---------+------+----------+------+------------+----+--------+--------------------+
only showing top 5 rows

### Due to missing data in the mpg and horsepower columns, they will be excluded from the dataset for model training.

In [11]:
%%spark -s $session_name
carsDataRaw = df_data_0
carsModData = carsDataRaw.drop("mpg").drop("horsepower")
carsModData.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+------------+----+--------+--------------------+
|cylinders|engine|weight|acceleration|year|  origin|                name|
+---------+------+------+------------+----+--------+--------------------+
|        8| 307.0|  3504|        12.0|  70|American|chevrolet chevell...|
|        8| 350.0|  3693|        11.5|  70|American|   buick skylark 320|
|        8| 318.0|  3436|        11.0|  70|American|  plymouth satellite|
|        8| 304.0|  3433|        12.0|  70|American|       amc rebel sst|
|        8| 302.0|  3449|        10.5|  70|American|         ford torino|
+---------+------+------+------------+----+--------+--------------------+
only showing top 5 rows

### In the model training process, the original dataset will be split into a training dataset and a testing dataset.

In [12]:
%%spark -s $session_name

splitted_data = carsModData.randomSplit([0.85, 0.15], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]

print("Number of training dataset: {}".format(train_data.count()))
print("Number of testing dataset: {}".format(test_data.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of training dataset: 348
Number of testing dataset: 58

### Now set the input columns for model training, and use the corresponding algorithms to train the model.
### In this example, the Linear Regression method is used to evaluate weight in the dataset.

In [13]:
%%spark -s $session_name
originIndexer = StringIndexer().setInputCol("origin").setOutputCol("origin_code")
vectorAssembler_features = VectorAssembler().setInputCols(
    ["cylinders", "engine", "acceleration", "year", "origin_code"]).setOutputCol("features")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
%%spark -s $session_name
rf = LinearRegression().setLabelCol("weight").setFeaturesCol("features")
pipeline = Pipeline().setStages([originIndexer,vectorAssembler_features,rf])
model = pipeline.fit(train_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Evaluate the Model
### The model performance can be evaluated using the R Square for test data.

In [15]:
%%spark -s $session_name
testData = model.transform(test_data).drop("prediction")
metric = model.stages[2].evaluate(testData)
print("R Square of Test Data: {}".format(metric.r2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R Square of Test Data: 0.863976844308

### Write the model to HDFS
### First, in the remote session, we use a Hadoop Integration utility method to write the model to HDFS.

In [16]:
%%spark -s $session_name
hi_core_utils.write_model_to_hdfs(model=model, model_name="ml_cars_model_pn_1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'path': 'hdfs:///user/user1/.dsxhi/models/ml_cars_model_pn_1/20/model', 'version': 20, 'name': 'ml_cars_model_pn_1', 'latest_version': 20}
Using TensorFlow backend.

### Download the model from HDFS to WS-Local

In [17]:
# Redeclare the dataset dir locally--the earlier declaration was in the _remote_
# session so it is not available here.

# ** NOTE ** Replace {your-username-here} with your actual user name.
hdfs_dataset_dir = "/user/user1/.dsxhi/models/ml_cars_model_pn_1/1/model.tar.gz"

# Download the model from HDFS to WS-Local.
#input_csv = os.environ["DSX_PROJECT_DIR"] + "/datasets/cars.csv"
#input_csv = "/project_data/data_asset/cars.csv"
wsl_model_path = "/project_data/data_asset/model.tar.gz"
hadoop_lib_utils.download_hdfs_file(webhdfs_endpoint, hdfs_dataset_dir, wsl_model_path)

download success


### Next use watson_machine_learning_client to process the model

In [18]:
%spark cleanup